In [ ]:
# Deep Research Agent
## 1. Clarifying questions when the query is vague
## 2. Multiple parallel searches
## 3. Report quality evaluation
## 4. Automatic retry if evaluation fails
## 5. Email delivery

In [ ]:
from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import requests
import os
from typing import Dict
from IPython.display import display, Markdown

In [ ]:
load_dotenv(override=True)

## Clarifier Agent

Asks clarifying questions when the query is vague or broad.

In [ ]:
class ClarificationOutput(BaseModel):
    needs_clarification: bool = Field(description="Whether the query needs clarification")
    questions: list[str] = Field(description="List of clarifying questions to ask the user")

CLARIFIER_INSTRUCTIONS = """You are a research clarification agent. Given a research query, decide if it needs clarification.

A query needs clarification if it is:
- Too broad or vague
- Missing key context like timeframe, location, or specific focus
- Ambiguous in intent

If clarification is needed, generate 2-3 specific questions that would help narrow the scope and improve research quality.
If the query is clear and specific enough, set needs_clarification to false and return empty questions list."""

clarifier_agent = Agent(
    name="ClarifierAgent",
    instructions=CLARIFIER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ClarificationOutput,
)

## Search Planner Agent

Creates a strategic plan of web searches to perform.

In [ ]:
HOW_MANY_SEARCHES = 5

class WebSearchItem(BaseModel):
    reason: str = Field(description="Why this search is important")
    query: str = Field(description="The search term to use")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="List of web searches to perform")

PLANNER_INSTRUCTIONS = f"""You are a research planning agent. Given a query, create {HOW_MANY_SEARCHES} strategic web searches to best answer it.

Make searches specific and complementary, covering different angles of the topic."""

planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

## Search Agent

Performs web searches and summarizes results.

In [ ]:
SEARCH_INSTRUCTIONS = """You are a research assistant. Given a search term, search the web and produce a concise summary.

Keep summaries to 2-3 paragraphs and under 300 words. Capture main points succinctly."""

search_agent = Agent(
    name="SearchAgent",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

## Report Writer Agent

Synthesizes search results into a comprehensive report.

In [ ]:
class ReportData(BaseModel):
    short_summary: str = Field(description="2-3 sentence summary of findings")
    markdown_report: str = Field(description="The full report in markdown")
    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

WRITER_INSTRUCTIONS = """You are a senior researcher writing a comprehensive report.

You will receive the original query and research summaries. Create a detailed report that:
- Provides clear structure and flow
- Synthesizes all findings cohesively
- Uses markdown formatting
- Is lengthy and detailed, aiming for 1000+ words
- Includes an executive summary, main findings, and conclusions"""

writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Report Evaluator Agent

Evaluates if the report meets quality standards.

In [ ]:
class EvaluationResult(BaseModel):
    is_acceptable: bool = Field(description="Whether the report meets quality standards")
    feedback: str = Field(description="Specific feedback on what needs improvement")

EVALUATOR_INSTRUCTIONS = """You are a report quality evaluator. Assess if the research report is:
- Comprehensive and well-structured
- Clearly written and easy to follow
- Sufficiently detailed (at least 800 words)
- Properly addresses the original query
- Contains actionable insights

Provide specific feedback on what needs improvement if the report is not acceptable."""

evaluator_agent = Agent(
    name="EvaluatorAgent",
    instructions=EVALUATOR_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=EvaluationResult,
)

## Email Agent Agent

Formats and sends the final report via email.

In [ ]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    mailgun_domain = os.getenv("MAILGUN_DOMAIN")
    mailgun_api_key = os.getenv("MAILGUN_API_KEY")
    from_email = os.getenv("MAILGUN_FROM_EMAIL", "noreply@" + mailgun_domain)
    to_email = os.getenv("MAILGUN_TO_EMAIL")
    
    response = requests.post(
        f"https://api.mailgun.net/v3/{mailgun_domain}/messages",
        auth=("api", mailgun_api_key),
        data={
            "from": from_email,
            "to": to_email,
            "subject": subject,
            "html": html_body
        }
    )
    
    return {"status": "success" if response.status_code == 200 else "failed"}

EMAIL_INSTRUCTIONS = """You are an email formatter. Convert the research report into a well-formatted HTML email.

Create a clean, professional HTML layout with appropriate subject line."""

email_agent = Agent(
    name="EmailAgent",
    instructions=EMAIL_INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

## Research Workflow Functions

In [ ]:
async def clarify_query(query: str):
    print("Checking if query needs clarification...")
    result = await Runner.run(clarifier_agent, f"Query: {query}")
    return result.final_output

async def plan_searches(query: str):
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    input = f"Search term: {item.query}\nReason: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

async def write_report(query: str, search_results: list[str], previous_feedback: str = ""):
    print("Writing report...")
    input = f"Original query: {query}\n\nSearch results: {search_results}"
    if previous_feedback:
        input += f"\n\nPrevious attempt was rejected. Feedback: {previous_feedback}\nPlease improve the report based on this feedback."
    result = await Runner.run(writer_agent, input)
    print("Report complete")
    return result.final_output

async def evaluate_report(query: str, report: str):
    print("Evaluating report quality...")
    input = f"Query: {query}\n\nReport:\n{report}"
    result = await Runner.run(evaluator_agent, input)
    evaluation = result.final_output
    if evaluation.is_acceptable:
        print("Report passed evaluation")
    else:
        print(f"Report needs improvement: {evaluation.feedback}")
    return evaluation

async def send_report(report: ReportData):
    print("Sending email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

## Main Research Agents Ochastration

In [ ]:
async def enhanced_deep_research(query: str, user_clarifications: str = ""):
    with trace("Enhanced Deep Research"):
        clarification = await clarify_query(query)
        
        if clarification.needs_clarification and not user_clarifications:
            print("\nQuery needs clarification. Please answer these questions:")
            for i, question in enumerate(clarification.questions, 1):
                print(f"{i}. {question}")
            print("\nRerun this function with your clarifications as the second parameter.")
            return
        
        final_query = query
        if user_clarifications:
            final_query = f"{query}\n\nAdditional context: {user_clarifications}"
        
        search_plan = await plan_searches(final_query)
        search_results = await perform_searches(search_plan)
        
        max_attempts = 2
        for attempt in range(max_attempts):
            if attempt == 0:
                report = await write_report(final_query, search_results)
            else:
                report = await write_report(final_query, search_results, evaluation.feedback)
            
            evaluation = await evaluate_report(final_query, report.markdown_report)
            
            if evaluation.is_acceptable:
                break
            
            if attempt == max_attempts - 1:
                print("Max attempts reached. Sending report anyway.")
        
        await send_report(report)
        
        display(Markdown(report.markdown_report))
        print("\nResearch complete!")

In [ ]:
query = "AI Agent frameworks"
clarifications = "Focus on production-ready frameworks, comparing OpenAI Agents SDK, LangGraph, and CrewAI. Looking for enterprise use cases in 2025."

await enhanced_deep_research(query, clarifications)